EDAS is an abbreviation for **Evaluation Based on Distance** from the Average Solution. It is one of the methods in multi-criteria decision making.

# PREPARATION

## Import Required Dependency

In [1]:
import math
from itertools import permutations
from itertools import product
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display
import json

## Global Helper Function

In [2]:
def create_criteria_decision_form() -> None: 
    global total_criteria
    
    dropdowns = []
    for i in range(total_criteria):
        label = widgets.Label(value=f"C{i+1}", layout=widgets.Layout(width="50px"))
    
        dropdown = widgets.Dropdown(options=['Benefit', 'Cost'], layout=widgets.Layout(width="100px"))
        dropdowns.append(dropdown)
    
        display(widgets.HBox([label, dropdown]))

    def get_dropdown_values():
        return [dropdown.value for dropdown in dropdowns]
    
    output = widgets.Output()
    
    def set_criteria(_):
        global criterias
        
        criterias[:] = get_dropdown_values()
            
        with output:
            output.clear_output(wait=True)
            print('Criterias')
            print(criterias)
        
    set_criteria_btn = widgets.Button(description="Set Criteria!")
    set_criteria_btn.on_click(set_criteria)

    display(set_criteria_btn, output)

In [3]:
def create_matrix_form() -> None:
    global total_criteria, total_alternative, criteria_labels, alternative_labels

    inputs = []
    for i in range(total_alternative):
        rows = []
        for c in range(total_criteria):
            rows.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
        inputs.append(rows)

    header_row = []
    for i in range(len(criteria_labels) + 1):
        header_row.append(widgets.Label(
            value='' if i == 0 else criteria_labels[i - 1],
            layout=widgets.Layout(width="100px")
        ))
    
    input_rows = []
    for r in range(total_alternative):
        row_widgets = [widgets.Label(value=alternative_labels[r], layout=widgets.Layout(width="50px"))] + inputs[r]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return np.array([[cell.value for cell in row] for row in inputs])

    output = widgets.Output()
    
    def update_matrix(_):
        global decision_matrix
        
        decision_matrix.loc[:, :] = get_matrix()

        with output:
            print("Decision matrix:")
            print(decision_matrix)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    display(widgets.HBox(header_row))
    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

In [4]:
def create_matrix_weight() -> None:
    global total_criteria, criteria_labels

    inputs = []
    for i in range(total_criteria):
        inputs.append(widgets.FloatText(value=0, layout=widgets.Layout(width="100px")))
    
    input_rows = []
    for r in range(total_criteria):
        row_widgets = [widgets.Label(value=criteria_labels[r], layout=widgets.Layout(width="100px"))] + [inputs[r]]
        input_rows.append(widgets.HBox(row_widgets))

    def get_matrix():
        return [row.value for row in inputs]

    output = widgets.Output()
    
    def update_matrix(_):
        global weights
        
        weights[:] = get_matrix()

        with output:
            print("Weight Criteria:")
            print(weights)

    set_matrix_btn = widgets.Button(description="Set Matrix!")
    set_matrix_btn.on_click(update_matrix)

    for row in input_rows:
        display(row)
        
    display(set_matrix_btn, output)

---

# Input Matrix-Like Data Structure of Criterias and Alternatives

In [5]:
total_criteria = 5
total_alternative = 5

criteria_labels = tuple(f"C{i + 1}" for i in range(total_criteria))
alternative_labels = tuple(f"A{i + 1}" for i in range(total_alternative))

criterias = pd.Series(None, index=criteria_labels)

Define criteria is benefit or cost

In [6]:
create_criteria_decision_form()

Button(description='Set Criteria!', style=ButtonStyle())

Output()

Define Weight for Each Criteria

In [7]:
weights = pd.Series(0, index=criteria_labels)

In [8]:
create_matrix_weight()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

In [9]:
decision_matrix = pd.DataFrame(0, index=alternative_labels, columns=criteria_labels)

In [10]:
create_matrix_form()

Button(description='Set Matrix!', style=ButtonStyle())

Output()

# Calculate Average Solution (AV)

Formula:

$$
AV_j = \frac{\Sigma_{i=1}^m X_{ij}}{m}
$$

In [11]:
av = pd.Series([decision_matrix[col].sum() / len(decision_matrix) for col in decision_matrix], index=criteria_labels)
av

C1    0.0
C2    0.0
C3    0.0
C4    0.0
C5    0.0
dtype: float64

# Calculate Positive Distance Criteria (PDA) and Negative Distance Criteria (NDA)

## PDA

Formula:

**Positive criteria**
$$
PDA_{ij} =
\frac{max(0, (x_{ij} - AV_j))}{AV_j}
$$

**Negative criteria**
$$
PDA_{ij} =
\frac{max(0, (AV_j - x_{ij}))}{AV_j}
$$

In [12]:
pda_data = {label: [] for label in criteria_labels}

for i in decision_matrix:
    for j in decision_matrix[i]:
        if criterias[i] == 'Benefit':
            value = max(0, (j - av[i])) / av[i]
        else:
            value = max(0, (av[i] - j)) / av[i]
        pda_data[i].append(value)

<ipython-input-12-d2a7e30a0364>:8: RuntimeWarning: invalid value encountered in scalar divide
  value = max(0, (av[i] - j)) / av[i]


In [13]:
pda = pd.DataFrame(pda_data, index=alternative_labels, columns=criteria_labels)
pda

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,C1,C2,C3,C4,C5
A1,NaN,NaN,NaN,NaN,NaN
A2,NaN,NaN,NaN,NaN,NaN
A3,NaN,NaN,NaN,NaN,NaN
A4,NaN,NaN,NaN,NaN,NaN
A5,NaN,NaN,NaN,NaN,NaN


## NDA

Formula:

**Positive criteria**
$$
NDA_{ij} =
\frac{max(0, (AV_j - x_{ij}))}{AV_j}
$$

**Negative criteria**
$$
NDA_{ij} =
\frac{max(0, (x_{ij} - AV_j))}{AV_j}
$$

In [14]:
nda_data = {label: [] for label in criteria_labels}

for i in decision_matrix:
    for j in decision_matrix[i]:
        if criterias[i] == 'Benefit':
            value = max(0, (av[i] - j)) / av[i]
        else:
            value = max(0, (j - av[i])) / av[i]
        nda_data[i].append(value)

<ipython-input-14-40466cfe67dc>:8: RuntimeWarning: invalid value encountered in scalar divide
  value = max(0, (j - av[i])) / av[i]


In [15]:
nda = pd.DataFrame(nda_data, index=alternative_labels, columns=criteria_labels)
nda

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,C1,C2,C3,C4,C5
A1,NaN,NaN,NaN,NaN,NaN
A2,NaN,NaN,NaN,NaN,NaN
A3,NaN,NaN,NaN,NaN,NaN
A4,NaN,NaN,NaN,NaN,NaN
A5,NaN,NaN,NaN,NaN,NaN


# Calculate Sum Weight of PDA and NDA (SP and SN)

SP is for PDA and SN is for NDA

Formula:

$$
SP_i = \Sigma_{j=1}^n w_j \times PDA_{ij}
$$

$$
SN_i = \Sigma_{j=1}^n w_j \times NDA_{ij}
$$

In [16]:
sp = pd.Series((pda * weights).sum(axis=1))
sp

A1    0.0
A2    0.0
A3    0.0
A4    0.0
A5    0.0
dtype: float64

In [17]:
sn = pd.Series((nda * weights).sum(axis=1))
sn

A1    0.0
A2    0.0
A3    0.0
A4    0.0
A5    0.0
dtype: float64

# Normalization SP and SN

NSP is for SP ans NSN is for SN.

Formula:

$$
NSP_i = \frac{SP_i}{max(SP_i)}
$$

$$
NSN_i = 1 - \frac{SN_i}{max(SN_i)}
$$

In [18]:
nsp = pd.Series(sp / sp.max())
nsp

/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.10/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


A1   NaN
A2   NaN
A3   NaN
A4   NaN
A5   NaN
dtype: float64

In [19]:
nsn = pd.Series(1 - (sn / sn.max()))
nsn

A1   NaN
A2   NaN
A3   NaN
A4   NaN
A5   NaN
dtype: float64

# Calculate Aprasial Score (AS)

Formula:

$$
AS_i = \frac{1}{2} (NSP_i + NSN_i)
$$

In [20]:
aprasial_score = 0.5 * (nsp + nsn)
aprasial_score

A1   NaN
A2   NaN
A3   NaN
A4   NaN
A5   NaN
dtype: float64

# Rangking The Alternatives

In [21]:
final_rank = aprasial_score.sort_values(ascending=False)
final_rank

A1   NaN
A2   NaN
A3   NaN
A4   NaN
A5   NaN
dtype: float64

In [22]:
first_index, first_value = next(final_rank.items())
print(f"So, the first rank is {first_index} with value: {first_value}")

So, the first rank is A1 with value: nan
